# Use Glove in Pytorch to Finish NLP task

Author:Huihan Yang

# Preparation

In [2]:
%cd ..

d:\杨蕙菡\assignment-2-text-classification-foxintohumanbeing


In [3]:
import torchtext
import os
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset
from torchtext.data import get_tokenizer
from torchtext.vocab import GloVe
from torch.nn.utils.rnn import pad_sequence
from torch import nn
import pandas as pd
import argparse
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

## Set random seed

In [4]:
seed = 114
np.random.seed(seed)
torch.manual_seed(seed)

# Hyperparameter

In [5]:
configs = {
    'work_dir': 'work_dir2', 
    'device': 'cuda:0',
    'batch': 32, 
    'optimizer_config': {
        'lr': 1e-4, 
    }, 
    'epoch': 100, 
    'model_name': 'bert-large-uncased',
    'enable_tb': False,
    'dropout':0.5
}

In [6]:
GLOVE_DIM = 100
GLOVE = GloVe(name='6B', dim=GLOVE_DIM)

# DataLoader

In [7]:
class TWITTERDataset(Dataset):

    def __init__(self, fname, is_train=True):
        super().__init__()
        self.tokenizer = get_tokenizer('basic_english')
        self.train = is_train
        if is_train == True:
            df = pd.read_csv(fname).iloc[:,1:]
        else:
            df = pd.read_csv(fname)
        self.lines = []
        for i in range(len(df)):
            if is_train == True:
                self.lines.append((df.iloc[i, 0], df.iloc[i, 1], df.iloc[i, 2], GLOVE.get_vecs_by_tokens(self.tokenizer(df.iloc[i, 3])), torch.tensor(df.iloc[i, 4], dtype=torch.int32) ))
            else:
                self.lines.append(( GLOVE.get_vecs_by_tokens(self.tokenizer(df.iloc[i, 3]))))
        print('Complete data preprocessing with length:', len(self.lines)) 

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, index: int):
        item = self.lines[index]
        if self.train:
            return item[3], item[4]
        return item


def get_dataloader():
    def collate_fn1(batch):
        x, y = zip(*batch)
        x_pad = pad_sequence(x, batch_first=True)
        y = torch.Tensor(y)
        return x_pad, y

    def collate_fn2(batch):
        x = [item for item in batch]
        x_pad = pad_sequence(x, batch_first=True)
        return x_pad

    train_dataloader = DataLoader(TWITTERDataset('nlp-getting-started/train_clean.csv'),
                    batch_size = configs['batch'],
                    shuffle = True,
                    collate_fn = collate_fn1)
    val_dataloader = DataLoader(TWITTERDataset('nlp-getting-started/val_clean.csv'),
                    batch_size = configs['batch'],
                    shuffle = True,
                    collate_fn = collate_fn1)
    test_dataloader = DataLoader(TWITTERDataset('nlp-getting-started/test.csv', False),
                    batch_size = configs['batch'],
                    shuffle = False,
                    collate_fn = collate_fn2)
    return train_dataloader,val_dataloader, test_dataloader


train_dataloader,val_dataloader, test_dataloader = get_dataloader()


Complete data preprocessing with length: 5329
Complete data preprocessing with length: 2284
Complete data preprocessing with length: 3263


# Define Model

In [8]:
device = configs['device']
class RNN(torch.nn.Module):
    def __init__(self, hidden_units=64, dropout_rate=configs['dropout']):
        super().__init__()
        self.drop = nn.Dropout(dropout_rate)
        self.rnn = nn.GRU(GLOVE_DIM, hidden_units, 1, batch_first=True)
        self.linear = nn.Linear(hidden_units, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x: torch.Tensor):
        # x shape: [batch, max_word_length, embedding_length]
        emb = self.drop(x)
        output, _ = self.rnn(emb)
        output = output[:, -1]
        output = self.linear(output)
        output = self.sigmoid(output)
        return output

model = RNN().to(device)

# Trainning

In [12]:
import os
os.getcwd()

'd:\\杨蕙菡\\assignment-2-text-classification-foxintohumanbeing'

In [13]:
writer = SummaryWriter('1141514')
optimizer = torch.optim.Adam(model.parameters(), lr=configs['optimizer_config']['lr'])
citerion = torch.nn.BCELoss()
os.makedirs('work_dir', exist_ok=True)
best_accuracy = 0
for epoch in range(configs['epoch']):
    model.train() # Set the model to training mode
    loss_sum = 0
    dataset_len = len(train_dataloader.dataset)
    for x, y in tqdm(train_dataloader):
        batchsize = y.shape[0]
        x = x.to(device)
        y = y.to(device)
        hat_y = model(x)
        hat_y = hat_y.squeeze(-1)
        loss = citerion(hat_y, y)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        loss_sum += loss * batchsize
    writer.add_scalar('training loss',
                            loss_sum/dataset_len,
                            epoch)

    model.eval()  # Set the model to evaluation mode
    correct_predictions = 0
    total_predictions = 0
    test_loss = 0
    results_predict = []
    with torch.no_grad():
        for x, y in tqdm(val_dataloader):
            x = x.to(device)
            y = y.to(device)

            hat_y = model(x)
            hat_y = hat_y.squeeze(-1)

            loss = citerion(hat_y, y)
            test_loss += loss.item() * y.size(0)

            # Calculate accuracy
            predictions = (hat_y > 0.5).int()  # Convert probabilities to binary predictions
            correct_predictions += (predictions == y).sum().item()
            total_predictions += y.size(0)
            results_predict.append(predictions.cpu())

    accuracy = correct_predictions / total_predictions
    avg_test_loss = test_loss / total_predictions
    writer.add_scalar('average validation accuracy', accuracy, epoch)

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        pt_path = os.path.join('baseline', 'best+original.pt')
        torch.save(model.state_dict(), pt_path)
        print('save model')
        results_predict = torch.concat(results_predict).tolist()
        id = pd.read_csv('nlp-getting-started/val_clean.csv')['id']
        prediction = pd.DataFrame()
        prediction['id'] = id.values
        prediction['target'] = results_predict
        prediction.to_csv('baseline/validation_result.csv',index=False)
        

    print(f'Epoch {epoch}. accuracy: {accuracy}')

100%|██████████| 72/72 [00:00<00:00, 1002.68it/s]


save model
Epoch 0. accuracy: 0.5647985989492119


100%|██████████| 72/72 [00:00<00:00, 759.93it/s]


Epoch 1. accuracy: 0.5647985989492119


100%|██████████| 72/72 [00:00<00:00, 880.40it/s]


...

Epoch 97. accuracy: 0.7972854640980735


100%|██████████| 72/72 [00:00<00:00, 772.17it/s]


Epoch 98. accuracy: 0.7937828371278459


100%|██████████| 72/72 [00:00<00:00, 849.33it/s]

Epoch 99. accuracy: 0.7898423817863398


# Inference

In [14]:
results_predict = []

model.load_state_dict(torch.load('baseline/best+original.pt'))
model.eval()
with torch.no_grad():
    for x in tqdm(test_dataloader):
            x = x.to(device)
            hat_y = model(x)
            hat_y = hat_y.squeeze(-1)
            predictions = (hat_y > 0.5).int() 
            results_predict.append(predictions.cpu())

results_predict = torch.concat(results_predict).tolist()

100%|██████████| 102/102 [00:00<00:00, 498.90it/s]


# Export the results


In [15]:
id = pd.read_csv('nlp-getting-started/test.csv')['id']
prediction = pd.DataFrame()
prediction['id'] = id.values
prediction['target'] = results_predict
prediction.to_csv('prediction_result/prediction_result_baseline.csv',index=False)